https://medium.com/bilesanmiahmad/how-to-upload-a-file-to-amazon-s3-in-python-68757a1867c6

## Upload images to S3 
Code from:  
https://boto3.amazonaws.com/v1/documentation/api/1.9.185/guide/s3-uploading-files.html

https://stackoverflow.com/questions/44978426/boto3-file-upload-does-it-check-if-file-exists

In [48]:
import boto3
import logging
from botocore.exceptions import ClientError

BUCKET_NAME = 'geoai-dtp-images'
ACCESS_KEY = 'ACCESS KEY'
SECRET_KEY = 'SECRET KEY'


# Function to upload files to S3 bucket 
def upload_to_s3(local_file, bucket_name, s3_file=None):
    
    # If S3 file name was note specified, use local filename 
    if s3_file is None:
        s3_file = local_file
    
    # Upload the file
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                   aws_secret_access_key=SECRET_KEY)
    try:
        s3.upload_file(local_file, bucket_name, s3_file)
    except ClientError as e:
        logging.error(e)
        return False
    return True


# Function to test existence of an object - to prevent uploading the same file
def check_s3_objects(bucket_name, key):
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                         aws_secret_access_key=SECRET_KEY)
    try:
        s3.head_object(Bucket=bucket_name, Key=key)
    except ClientError as e:
        return int(e.response['Error']['Code']) != 404
        #return False
    return True

### Upload image files from selected folder: 

In [25]:
from os import listdir
from os.path import isfile, join, split

folder = "C:\\Users\\steff\\Documents\\05 GeoAI\\03 AI\\GeoAI\\data\\01 Images"

# Create a list to store files names with paths 
files = [join(folder, f) for f in listdir(folder) if isfile(join(folder,f))]

In [59]:
# iterate through the files list and upload to our S3 Bucket if they dont exist yet
for f in files:
    # get file name without file path to use as S3 object key 
    path, file = split(f)
    if not check_s3_objects(BUCKET_NAME, f"images/{file}"):
        #upload to S3
        upload_to_s3(f, BUCKET_NAME, s3_file=f"images/{file}")

### Upload annotations files

In [62]:
annotations_folder = "C:\\Users\\steff\\Documents\\05 GeoAI\\03 AI\\GeoAI\\01_data\\output annotations"
annotations = [join(annotations_folder, a) 
               for a in listdir(annotations_folder)
              if isfile(join(annotations_folder, a))]

for a in annotations:
    path, file = split(a)
    upload_to_s3(a, BUCKET_NAME, s3_file = f"annotations/{file}")